In [1]:
# a) Load libraries
import os
import glob
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
import cv2
!pip install imagehash
import imagehash

import torchvision.models.segmentation
import torch
import torchvision.transforms as tf

from sklearn.model_selection import train_test_split
from PIL import Image

from pickle import dump

# Check if GPU parallel computing is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.8 MB/s eta 0:00:00
cuda


In [2]:
%pip install geedim -q
%pip install torchgeo -q
%pip install rioxarray -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 690.3/690.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.2/535.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
import rioxarray
# Deep Learning libraries
import torch
import torchdata

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
from pathlib import Path
from torchgeo.datasets import RasterDataset, unbind_samples, stack_samples
from torchgeo.samplers import RandomBatchGeoSampler, RandomGeoSampler, Units
from torch.utils.data import DataLoader

In [6]:
from pathlib import Path
root = Path('/content/drive/My Drive/Colab Notebooks/dataset.zip/test_landcovers/')
assert root.exists()
print(root.as_posix())
test_landcovers = RasterDataset(paths=root.as_posix(), transforms=None)

root = Path('/content/drive/My Drive/Colab Notebooks/dataset.zip/test_masks')
assert root.exists()
print(root.as_posix())
test_mask = RasterDataset(paths=root.as_posix())
test_mask.is_image = False

from pathlib import Path
root = Path('/content/drive/My Drive/Colab Notebooks/dataset.zip/train_landcovers/')
assert root.exists()
print(root.as_posix())
train_landcovers = RasterDataset(paths=root.as_posix(), transforms=None)

root = Path('/content/drive/My Drive/Colab Notebooks/dataset.zip/train_masks')
assert root.exists()
print(root.as_posix())
train_mask = RasterDataset(paths=root.as_posix())
train_mask.is_image = False

from pathlib import Path
root = Path('/content/drive/My Drive/Colab Notebooks/dataset.zip/valid_landcovers/')
assert root.exists()
print(root.as_posix())
valid_landcovers = RasterDataset(paths=root.as_posix(), transforms=None)

root = Path('/content/drive/My Drive/Colab Notebooks/dataset.zip/valid_masks')
assert root.exists()
print(root.as_posix())
valid_mask = RasterDataset(paths=root.as_posix())
valid_mask.is_image = False


/content/drive/My Drive/Colab Notebooks/dataset.zip/test_landcovers
/content/drive/My Drive/Colab Notebooks/dataset.zip/test_masks
/content/drive/My Drive/Colab Notebooks/dataset.zip/train_landcovers
/content/drive/My Drive/Colab Notebooks/dataset.zip/train_masks
/content/drive/My Drive/Colab Notebooks/dataset.zip/valid_landcovers
/content/drive/My Drive/Colab Notebooks/dataset.zip/valid_masks


In [7]:
train_dset = train_landcovers & train_mask
valid_dset = valid_landcovers & valid_mask
test_dset = test_landcovers & test_mask

train_sampler = RandomGeoSampler(train_landcovers, size=512, length=260, units=Units.PIXELS)
valid_sampler = RandomGeoSampler(valid_landcovers, size=512, length=128, units=Units.PIXELS)
test_sampler = RandomGeoSampler(test_landcovers, size=512, length=128, units=Units.PIXELS)

In [8]:
train_dataloader = DataLoader(train_dset, sampler=train_sampler, batch_size=8, collate_fn=stack_samples)
valid_dataloader = DataLoader(valid_dset, sampler=valid_sampler, batch_size=8, collate_fn=stack_samples)
test_dataloader = DataLoader(test_dset, sampler=test_sampler, batch_size=8, collate_fn=stack_samples)

train_batch = next(iter(train_dataloader))
valid_batch = next(iter(valid_dataloader))
test_batch = next(iter(test_dataloader))
train_batch.keys(), valid_batch.keys()

(dict_keys(['crs', 'bbox', 'image', 'mask']),
 dict_keys(['crs', 'bbox', 'image', 'mask']))

# **Torchgeo Segmentation model tutorial**
https://github.com/microsoft/torchgeo

In [ ]:
import timm
from torchgeo.models import ResNet18_Weights

weights = ResNet18_Weights.SENTINEL2_ALL_MOCO
model = timm.create_model("resnet18", in_chans=weights.meta["in_chans"], num_classes=10)
model = model.load_state_dict(weights.get_state_dict(progress=True), strict=False)

Downloading: "https://huggingface.co/torchgeo/resnet18_sentinel2_all_moco/resolve/main/resnet18_sentinel2_all_moco-59bfdff9.pth" to /root/.cache/torch/hub/checkpoints/resnet18_sentinel2_all_moco-59bfdff9.pth
100%|██████████| 42.8M/42.8M [00:00<00:00, 150MB/s]


In [ ]:
import tempfile


accelerator = "gpu" if torch.cuda.is_available() else "cpu"
default_root_dir = os.path.join(tempfile.gettempdir(), "experiments")

In [ ]:
from lightning.pytorch import Trainer

trainer = Trainer(
    accelerator=accelerator,
    default_root_dir=default_root_dir,
    fast_dev_run=False,
    log_every_n_steps=1,
    min_epochs=1,
    max_epochs=10,
)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
!pip install torchgeo lightly!=1.4.26

# https://github.com/microsoft/torchgeo/issues/1824

In [ ]:
from torchgeo.trainers import SemanticSegmentationTask

task = SemanticSegmentationTask(
    model="unet",
    backbone="resnet50",
    weights=True,
    in_channels=3,
    num_classes=2,
    loss="ce",
    ignore_index=None,
    lr=0.1,
    patience=6,
)

trainer.fit(model=task, datamodule=dataloader)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 121MB/s]


AttributeError: 'DataLoader' object has no attribute 'prepare_data_per_node'

# **Segmentation of satellite images of water bodies**
https://github.com/tstran155/Segmentation-of-satellite-images-of-water-bodies/blob/main/Notebook2_satellite_image_segmentation_pytorch_final.ipynb

In [11]:
# a) Define pretrained ResNet-50 model using TorchVision DeepLabV3 architecture
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained = True)

# Since the segmented images have 2 classes (black and white), we change the final layer to 2 classes
model.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size = (1, 1), stride = (1, 1))
model = model.to(device)

# Select Stochastic gradient descent (SGD) optimizer with a learning rate of 0.01 (optimum for image segmentation)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:01<00:00, 117MB/s]


In [12]:
# Define loss function
criterion = torch.nn.CrossEntropyLoss()
Losses = []

In [16]:
batch = 10

def LoadBatch(df_images, df_masks):
    images = torch.zeros([batch, 3, height, width])
    masks = torch.zeros([batch, height, width])
    for i in range(batch):
        images[i], masks[i] = ReadImage(df_images, df_masks)
    return images, masks

In [17]:
# b) Evaluation of train dataset
epochs = 1200
X_train = train_lancovers
y_train = train_mask

# Training loop
for itr in range(epochs):
   images, masks = LoadBatch(X_train, y_train)
   # Load images and masks tensors to the GPU
   images = torch.autograd.Variable(images,requires_grad = False).to(device)
   masks = torch.autograd.Variable(masks, requires_grad = False).to(device)

   # Forward training loop
   Pred = model(images)['out']
   model.zero_grad()
   Loss = criterion(Pred, masks.long())
   Losses.append(Loss.item())

   # Backward training loop
   Loss.backward()
   optimizer.step() # Apply gradient descent to optimize weights
   seg = torch.argmax(Pred[0], 0).cpu().detach().numpy() # Move the tensor to the CPU, detach it from the graph, and convert it to a NumPy array
   print(itr,") Loss=", Loss.data.cpu().numpy())

NameError: name 'height' is not defined

In [ ]:
# Plot the training loss values
plt.plot(Losses)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
# a) Predictions on new images and masks
# Set trained model to the evaluation mode
model.eval()

# **Artificial-intelligence-for-geospatial-analysis**
https://www.geocorner.net/post/artificial-intelligence-for-geospatial-analysis-with-pytorch-s-torchgeo-part-3

In [9]:
from torchvision.models.segmentation import deeplabv3_resnet50
model = deeplabv3_resnet50(weights=None, num_classes=2)

model

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 127MB/s]


DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [10]:
import torch.nn as nn

backbone = model.get_submodule('backbone')

conv = nn.modules.conv.Conv2d(
    in_channels=9,
    out_channels=64,
    kernel_size=(7, 7),
    stride=(2, 2),
    padding=(3, 3),
    bias=False
)
backbone.register_module('conv1', conv)

pred = model(torch.randn(3, 9, 512, 512))
pred['out'].shape

torch.Size([3, 2, 512, 512])

In [11]:
from typing import Iterable, List, Callable, Optional

def train_loop(
    epochs: int,
    train_dl: DataLoader,
    val_dl: Optional[DataLoader],
    model: nn.Module,
    loss_fn: Callable,
    optimizer: torch.optim.Optimizer,
    acc_fns: Optional[List]=None,
    batch_tfms: Optional[Callable]=None
):
    # size = len(dataloader.dataset)
    cuda_model = model.cuda()

    for epoch in range(epochs):
        accum_loss = 0
        for batch in train_dl:

            if batch_tfms is not None:
                batch = batch_tfms(batch)

            X = batch['image'].cuda()
            y = batch['mask'].type(torch.long).cuda()
            pred = cuda_model(X)['out']
            loss = loss_fn(pred, y)

            # BackProp
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update the accum loss
            accum_loss += float(loss) / len(train_dl)

        # Testing against the validation dataset
        if acc_fns is not None and val_dl is not None:
            # reset the accuracies metrics
            acc = [0.] * len(acc_fns)

            with torch.no_grad():
                for batch in val_dl:

                    if batch_tfms is not None:
                        batch = batch_tfms(batch)

                    X = batch['image'].type(torch.float32).cuda()
                    y = batch['mask'].type(torch.long).cuda()

                    pred = cuda_model(X)['out']

                    for i, acc_fn in enumerate(acc_fns):
                        acc[i] = float(acc[i] + acc_fn(pred, y)/len(val_dl))

            # at the end of the epoch, print the errors, etc.
            print(f'Epoch {epoch}: Train Loss={accum_loss:.5f} - Accs={[round(a, 3) for a in acc]}')
        else:

            print(f'Epoch {epoch}: Train Loss={accum_loss:.5f}')

In [12]:
from sklearn.metrics import jaccard_score

def oa(pred, y):
    flat_y = y.squeeze()
    flat_pred = pred.argmax(dim=1)
    acc = torch.count_nonzero(flat_y == flat_pred) / torch.numel(flat_y)
    return acc

def iou(pred, y):
    flat_y = y.cpu().numpy().squeeze()
    flat_pred = pred.argmax(dim=1).detach().cpu().numpy()
    return jaccard_score(flat_y.reshape(-1), flat_pred.reshape(-1), zero_division=1.)

def loss(p, t):
    return torch.nn.functional.cross_entropy(p, t.squeeze())


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)
train_loop(10, train_dataloader, valid_dataloader, model, loss, optimizer,
           acc_fns=[oa, iou])

RuntimeError: Given groups=1, weight of size [64, 9, 7, 7], expected input[8, 3, 512, 512] to have 9 channels, but got 3 channels instead